# Differentiable Neural Computer

<a href="http://www.nature.com/nature/journal/v538/n7626/full/nature20101.html"><img src="./static/dnc_schema.png" alt="DNC schema" style="width: 900px;"/></a>

**Sam Greydanus $\cdot$ February 2017 $\cdot$ MIT License.**

Represents the state of the art in differentiable memory. Inspired by this [Nature paper](http://www.nature.com/nature/journal/v538/n7626/full/nature20101.html). Some ideas taken from [this Gihub repo](https://github.com/Mostafa-Samir/DNC-tensorflow)

In [1]:
import tensorflow as tf
import numpy as np
import sys

from dnc.dnc import DNC
from nn_controller import NNController

In [2]:
length = 10
xydim = 6
tf.app.flags.DEFINE_integer("xlen", xydim, "Input dimension")
tf.app.flags.DEFINE_integer("ylen", xydim, "output dimension")
tf.app.flags.DEFINE_integer("max_sequence_length", 2*length+1, "Maximum sequence length")
tf.app.flags.DEFINE_integer("length", length, "Maximum sequence length")
tf.app.flags.DEFINE_integer("batch_size", 2, "Size of batch in minibatch gradient descent")

tf.app.flags.DEFINE_integer("R", 1, "Number of DNC read heads")
tf.app.flags.DEFINE_integer("W", 10, "Word length for DNC memory")
tf.app.flags.DEFINE_integer("N", 15, "Number of words the DNC memory can store")

tf.app.flags.DEFINE_integer("print_every", 100, "Print training info after this number of train steps")
tf.app.flags.DEFINE_integer("iterations", 100000, "Number of training iterations")
tf.app.flags.DEFINE_float("lr", 1e-4, "Learning rate (alpha) for the model")
tf.app.flags.DEFINE_float("momentum", .9, "Momentum for RMSProp")
tf.app.flags.DEFINE_integer("save_every", 0, "Save model after this number of train steps")
tf.app.flags.DEFINE_string("save_dir", "models", "Directory in which to save checkpoints")
tf.app.flags.DEFINE_string("log_dir", "logs", "Directory in which to save logs")
FLAGS = tf.app.flags.FLAGS

In [3]:
def generate_data(batch_size, length, dim):
    X, y = np.zeros((batch_size, 2 * length + 1, dim)), np.zeros((batch_size, 2 * length + 1, dim))
    sequence = np.random.binomial(1, 0.5, (batch_size, length, dim - 1))

    X[:, :length, :dim - 1] = sequence
    X[:, length, -1] = 1  # end symbol
    y[:, length + 1:, :dim - 1] = sequence
    
    return X, y

def binary_cross_entropy(y_hat, y):
    return tf.reduce_mean(-y*tf.log(y_hat) - (1-y)*tf.log(1-y_hat))

def llprint(message):
    sys.stdout.write(message)
    sys.stdout.flush()

In [4]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session(graph=graph) as session:
        llprint("building graph...\n")
        optimizer = tf.train.RMSPropOptimizer(FLAGS.lr, momentum=FLAGS.momentum)
        dnc = DNC(NNController, FLAGS)

        # define loss
        y_hat, _ = dnc.get_outputs()
        y_hat = tf.clip_by_value(tf.sigmoid(y_hat), 1e-6, 1. - 1e-6)
        loss = binary_cross_entropy(y_hat, dnc.y)
        
        llprint("computing gradients...\n")
        gradients = optimizer.compute_gradients(loss)
        grad_op = optimizer.apply_gradients(gradients)
    
        llprint("init variables... \n")
        session.run(tf.global_variables_initializer())
        llprint("starting to train...\n\n")

        loss_history = []

        for i in xrange(FLAGS.iterations + 1):
            llprint("\rIteration {}/{}".format(i, FLAGS.iterations))

            random_length = np.random.randint(1, FLAGS.length + 1)
            X, y = generate_data(FLAGS.batch_size, random_length, FLAGS.xlen)

            fetch = [loss, grad_op]
            feed = {dnc.X: X, dnc.y: y, dnc.tsteps: 2 * random_length + 1}
            
            step_loss, _ = session.run(fetch, feed_dict=feed)

            loss_history.append(step_loss)

            if i % 100 == 0:
                llprint("\n\tloss: {:03.4f}\n".format(np.mean(loss_history)))
                loss_history = []

building graph...
computing gradients...
init variables... 
starting to train...

Iteration 0/100000
	loss: 0.6899
Iteration 100/100000
	loss: 0.6738
Iteration 200/100000
	loss: 0.4000
Iteration 300/100000
	loss: 0.2642
Iteration 400/100000
	loss: 0.2544
Iteration 500/100000
	loss: 0.2533
Iteration 600/100000
	loss: 0.2539
Iteration 700/100000
	loss: 0.2570
Iteration 800/100000
	loss: 0.2507
Iteration 900/100000
	loss: 0.2462
Iteration 1000/100000
	loss: 0.2464
Iteration 1100/100000
	loss: 0.2491
Iteration 1200/100000
	loss: 0.2412
Iteration 1300/100000
	loss: 0.2340
Iteration 1400/100000
	loss: 0.2343
Iteration 1500/100000
	loss: 0.2303
Iteration 1600/100000
	loss: 0.2196
Iteration 1700/100000
	loss: 0.2305
Iteration 1800/100000
	loss: 0.2237
Iteration 1900/100000
	loss: 0.2082
Iteration 2000/100000
	loss: 0.2180
Iteration 2100/100000
	loss: 0.2105
Iteration 2200/100000
	loss: 0.1964
Iteration 2300/100000
	loss: 0.1891
Iteration 2400/100000
	loss: 0.1780
Iteration 2500/100000
	loss: 0

KeyboardInterrupt: 